# Suspensions Over a Date Range for Dashboard

In this Jupyter notebook, the suspensions for students will be calculated and presented as an unduplicated count. A sheet will be generated that has the suspension rates for each school site and as well as another sheet that is showing all the students who have been suspended.

It will be based on two queries in Aeries:

Enrollment: LIST STU ID LN FN SC GR ED 

Suspension: LIST STU ADS DSP ADS.SC STU.ID ADS.DT ADS.CD DSP.DS DSP.DD

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Obtain enrollment data for the students
# LIST STU ID LN FN SC GR ED 

enrollment = pd.read_excel(r"C:\Users\derek.castleman\Desktop\MonthlyDataPull\2024-2025\October\Enrollment October.xlsx")

# Discipline data for the students
# LIST STU ADS DSP ADS.SC STU.ID ADS.DT ADS.CD DSP.DS DSP.DD
suspension = pd.read_excel(r"C:\Users\derek.castleman\Desktop\MonthlyDataPull\2024-2025\October\SuspensionsData.xlsx")

In [ ]:
a = input('What is the start date you are interested in (mm/dd/yyyy):          ') #Input start date

In [ ]:
a = pd.to_datetime(a) # Change start date to datetime
a

In [ ]:
b = input('What is the end date you are interested in (mm/dd/yyyy):          ') #Input end date

In [ ]:
b = pd.to_datetime(b) # Turn end date to date time
b

In [ ]:
suspension

In [ ]:
suspension['Disposition date']= pd.to_datetime(suspension['Disposition date']) # Changes disposition date to datetime
suspension

In [ ]:
# Selects dispositions between the two dates
dates_interested = suspension[(suspension['Disposition date'] >=a) & (suspension['Disposition date'] <=b)]
dates_interested

In [ ]:
# Selection only the suspensions from among the dispositions
suspension = dates_interested[(dates_interested['Disposition'] == 'SUS-13')
                              | (dates_interested['Disposition'] == '04')]
suspension

In [ ]:
# Gives a value of 1 to each suspension
suspension['Suspensions'] = 1
suspension

In [ ]:
m = input('Duplicated or Unduplicated:          ')
m

In [ ]:
# Based on what was input duplicated or unduplicated counts will be selected
if m == 'Unduplicated':
    suspension = suspension.drop_duplicates(subset=['Student ID'])
    suspension_students = suspension
    suspension
else: suspension_students = suspension.groupby(by=['School', 'Student ID'])['Suspensions'].sum().reset_index()

In [ ]:
suspension_students

In [ ]:
# Selects columns of interest
suspension = suspension[['School', 'Suspensions']]
suspension

In [ ]:
# Sums up the suspensions by school
suspension = suspension.groupby(by=['School'])['Suspensions'].sum().reset_index()
suspension

In [ ]:
enrollment
enrollment_students = enrollment

In [ ]:
# Giving each student an enrollment number
enrollment['Enrollment'] = 1
enrollment

In [ ]:
# Summing up the enrollment by school
enrollment = enrollment.groupby(by=['School'])['Enrollment'].sum().reset_index()
enrollment

In [ ]:
# Merge enrollment with the suspension tables
school = pd.merge(enrollment, suspension, how='outer', on='School')
school

In [ ]:
# Fills in with zero if there are none there
school = school.fillna(0)
school

In [ ]:
# Putting the secondary schools together
delano_ss = school[(school['School'] == 1) | (school['School'] == 2)]
delano_ss

In [ ]:
# Summing all columns
column_sums = delano_ss.sum(axis=0)
delano_ss = pd.DataFrame(column_sums).transpose()
delano_ss

In [ ]:
# Renaming to the SS
delano_ss['School'] = delano_ss['School'].replace(3.0, 'Delano SS')
delano_ss

In [ ]:
# Selecting for all of Delano
delano = school[(school['School'] == 1) | (school['School'] == 2) | (school['School'] == 4) ]
delano

In [ ]:
# Adding up columns
column_sums = delano.sum(axis=0)
delano = pd.DataFrame(column_sums).transpose()
delano

In [ ]:
# Rename for Delano
delano['School'] = delano['School'].replace(7.0, 'Delano')
delano

In [ ]:
# Secondary for Lost Hills is selected
lh_ss = school[(school['School'] == 7) | (school['School'] == 8) ]
lh_ss

In [ ]:
# Summing up columns
column_sums = lh_ss.sum(axis=0)
lh_ss = pd.DataFrame(column_sums).transpose()
lh_ss

In [ ]:
# Renaming it
lh_ss['School'] = lh_ss['School'].replace(15.0, 'Lost Hills SS')
lh_ss

In [ ]:
# Selecting Lost Hills
lh = school[(school['School'] == 6) | (school['School'] == 7) | (school['School'] == 8) ]
lh

In [ ]:
# Summing up columns
column_sums = lh.sum(axis=0)
lh = pd.DataFrame(column_sums).transpose()
lh

In [ ]:
# Renaming the columns
lh['School'] = lh['School'].replace(21.0, 'Lost Hills')
lh

In [ ]:
# Put all the aggregated schools together with the schools
school = pd.concat([school, delano_ss, delano, lh_ss, lh])
school

In [ ]:
# Replacing the schools with their names
replacement_dict = {1.0: 'Delano HS', 2: 'Delano MS', 4.0:'Delano ES', 6.0:'Lost Hills ES', 
                   7.0:'Lost Hills MS', 8.0: 'Lost Hills HS'}
school['School'].replace(replacement_dict, inplace=True)
school

In [ ]:
# Calculating the suspenion rate
school['Supension Rate'] = school['Suspensions'] / school['Enrollment']
school

In [ ]:
suspension_students

In [ ]:
enrollment_students

In [ ]:
suspended_students = pd.merge(enrollment_students, suspension_students, how='left', on='Student ID')
suspended_students

In [ ]:
suspended_students['Suspensions'].fillna(0, inplace=True)
suspended_students

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Suspensions", filename = "Suspensions"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(school)

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Suspension by Student", filename = "Suspension by Student"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(suspended_students)